# **Paso 1: Encontrar y Cargar los Archivos (con glob)**

En este paso, usamos la librería `glob` para encontrar automáticamente todos los archivos CSV que comienzan con "ventas_". Esto es muy útil cuando tienes múltiples archivos y no quieres escribir cada nombre manualmente.

## ¿Qué es glob?
- **glob** permite buscar archivos usando patrones (wildcards)
- El asterisco `*` significa "cualquier cosa"
- Por ejemplo: `ventas_*.csv` encuentra: `ventas_semana_1.csv`, `ventas_semana_2.csv`, etc.

## Proceso:
1. Definimos la ruta con patrón
2. Encontramos todos los archivos que coinciden
3. Cargamos cada uno en un DataFrame
4. Agregamos una columna para rastrear el origen de cada dato

In [5]:
# ============================================================================
# IMPORTAR LIBRERÍAS NECESARIAS
# ============================================================================
import pandas as pd  # Para trabajar con datos en tablas (DataFrames)
import glob          # Para buscar archivos usando patrones (wildcards)
import os            # Para operaciones del sistema (nombres de archivos, etc)

# ============================================================================
# PASO 1: DEFINIR LA RUTA DE BÚSQUEDA
# ============================================================================
# Patrón: "ventas_*.csv"
# - Los archivos están en la carpeta actual (project2)
# - El asterisco (*) significa "cualquier cosa"
# - Esto encontrará: ventas_semana_1.csv, ventas_semana_2.csv, etc.
ruta_archivos = "ventas_*.csv"

# ============================================================================
# PASO 2: USAR GLOB PARA ENCONTRAR LOS ARCHIVOS
# ============================================================================
# glob.glob() retorna una lista de rutas que coinciden con el patrón
# sorted() ordena la lista alfabéticamente para consistencia
lista_archivos_csv = sorted(glob.glob(ruta_archivos))

print("Archivos encontrados:")
for archivo in lista_archivos_csv:
    print(f"  - {archivo}")

# ============================================================================
# PASO 3: CREAR UNA LISTA VACÍA PARA ALMACENAR LOS DATAFRAMES
# ============================================================================
# lista_dataframes será un "contenedor" para cada tabla (DataFrame) que cargemos
lista_dataframes = []

# ============================================================================
# PASO 4: ITERAR SOBRE CADA ARCHIVO Y CARGARLO
# ============================================================================
# Para cada archivo encontrado:
for archivo in lista_archivos_csv:
    try:  # Intentamos cargar el archivo
        # pd.read_csv() lee el archivo CSV y lo convierte en un DataFrame (tabla)
        df_temporal = pd.read_csv(archivo)
        
        # Buena práctica: agregamos una columna para identificar el origen
        # Si luego hay errores, sabremos de cuál archivo vino cada dato
        df_temporal['FuenteArchivo'] = os.path.basename(archivo)
        
        # Agregamos este DataFrame a nuestra lista
        lista_dataframes.append(df_temporal)
        
        # Mostramos un mensaje de éxito
        print(f"✓ {archivo} cargado correctamente ({len(df_temporal)} filas)")
        
    except Exception as e:  # Si hay un error...
        # Mostramos cuál archivo tuvo problema y qué error fue
        print(f"✗ Error al cargar {archivo}: {e}")

# ============================================================================
# PASO 5: RESUMEN
# ============================================================================
print(f"\nSe cargaron {len(lista_dataframes)} archivos exitosamente.")

Archivos encontrados:
  - ventas_semana_1.csv
  - ventas_semana_2.csv
  - ventas_semana_3.csv
✓ ventas_semana_1.csv cargado correctamente (5 filas)
✓ ventas_semana_2.csv cargado correctamente (5 filas)
✓ ventas_semana_3.csv cargado correctamente (5 filas)

Se cargaron 3 archivos exitosamente.


# **Paso 2: Consolidar los Datos (con pd.concat)**

Ahora que tenemos todos los archivos cargados en DataFrames separados, los "pegamos" en uno solo.

## ¿Por qué consolidar?
- Tenemos 3 archivos separados (semana 1, 2 y 3)
- Para el análisis, necesitamos todos los datos en una sola tabla
- `pd.concat()` es como un "copiar y pegar" en Excel

## Parámetro importante:
- `ignore_index=True`: Resetea los índices para evitar duplicados
  - Ejemplo: Semana 1 tiene filas 0-4, Semana 2 tiene filas 0-4
  - Sin esto, el resultado tendría índices duplicados
  - Con esto, tenemos filas 0-14 en orden

In [6]:
# ============================================================================
# CONSOLIDAR TODOS LOS DATAFRAMES EN UNO SOLO
# ============================================================================
# pd.concat() "pega" todos los DataFrames de la lista en uno solo
# Es como hacer Ctrl+C y Ctrl+V en Excel, pero automático
df_consolidado = pd.concat(lista_dataframes, ignore_index=True)

# Explicación de ignore_index=True:
# - Sin él: tendríamos índices duplicados (0,1,2,3,4,0,1,2,3,4,...)
# - Con él: tenemos índices únicos (0,1,2,3,4,5,6,7,8,...)

# ============================================================================
# VISUALIZAR LOS DATOS CONSOLIDADOS
# ============================================================================
print("\n--- Datos Consolidados (Maestro) ---")
print(f"Total de filas: {len(df_consolidado)}")
print(f"Total de columnas: {len(df_consolidado.columns)}")
print("\nPrimeras 10 filas:")
print(df_consolidado.head(10))

# Información adicional útil:
print("\n--- Información del DataFrame ---")
print(df_consolidado.info())  # Muestra tipos de datos y valores nulos


--- Datos Consolidados (Maestro) ---
Total de filas: 15
Total de columnas: 5

Primeras 10 filas:
        Fecha Producto  Cantidad  PrecioUnitario        FuenteArchivo
0  2025-11-03   Laptop         5            1200  ventas_semana_1.csv
1  2025-11-04    Mouse        10              25  ventas_semana_1.csv
2  2025-11-05  Teclado         8              70  ventas_semana_1.csv
3  2025-11-06  Monitor         3             300  ventas_semana_1.csv
4  2025-11-07   Webcam         6              80  ventas_semana_1.csv
5  2025-11-10   Laptop         3            1200  ventas_semana_2.csv
6  2025-11-11  Monitor         5             300  ventas_semana_2.csv
7  2025-11-12    Mouse        12              25  ventas_semana_2.csv
8  2025-11-13  Teclado         7              70  ventas_semana_2.csv
9  2025-11-14   Webcam         4              80  ventas_semana_2.csv

--- Información del DataFrame ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 0 to 14
Data columns (total 5 colum

# **Paso 3: Analizar y Resumir**

Ahora vamos a procesar los datos para:
1. Calcular el ingreso total por venta (Cantidad × Precio Unitario)
2. Agrupar y sumar por producto
3. Calcular el total general de ingresos

## Concepto: groupby() - Agrupar y Resumir
- `groupby('Producto')`: Agrupa todos los registros por producto
- `agg()`: Aplica funciones de agregación (sum, count, mean, etc.)
- `reset_index()`: Convierte el resultado a un DataFrame normal

## Resultado esperado:
Una tabla con cada producto y sus totales de unidades e ingresos

In [7]:
# ============================================================================
# ANÁLISIS 1: CREAR COLUMNA DE INGRESO TOTAL
# ============================================================================
# Fórmula simple: Cantidad × Precio Unitario = Ingreso Total
# Ejemplo: 5 unidades × $1200 = $6000
df_consolidado['IngresoTotal'] = df_consolidado['Cantidad'] * df_consolidado['PrecioUnitario']

print("\n--- Datos con Ingresos Calculados (primeras 5 filas) ---")
print(df_consolidado[['Fecha', 'Producto', 'Cantidad', 'PrecioUnitario', 'IngresoTotal']].head())

# ============================================================================
# ANÁLISIS 2: AGRUPAR Y RESUMIR POR PRODUCTO
# ============================================================================
# groupby('Producto'): Agrupa todos los registros por producto
# agg(): Aplica funciones de agregación
#   - 'sum': suma todos los valores en el grupo
#   - Podemos usar también: 'mean' (promedio), 'count' (cantidad), 'min', 'max', etc.

resumen_productos = df_consolidado.groupby('Producto').agg(
    TotalUnidadesVendidas=('Cantidad', 'sum'),      # Suma de cantidades por producto
    TotalIngresos=('IngresoTotal', 'sum')           # Suma de ingresos por producto
).reset_index()  # reset_index() convierte esto en un DataFrame normal

print("\n--- Resumen por Producto (para Gerencia) ---")
print(resumen_productos)

# ============================================================================
# ANÁLISIS 3: CALCULAR EL GRAN TOTAL
# ============================================================================
# Sumamos todos los ingresos de todos los productos
total_general = resumen_productos['TotalIngresos'].sum()

# Mostramos el resultado formateado como dinero
print(f"\n{'='*50}")
print(f"Ingreso Total General: ${total_general:,.2f}")
print(f"{'='*50}")

# Información adicional útil:
print(f"\nProducto con mayor ingreso: {resumen_productos.loc[resumen_productos['TotalIngresos'].idxmax(), 'Producto']}")
print(f"Producto con menor ingreso: {resumen_productos.loc[resumen_productos['TotalIngresos'].idxmin(), 'Producto']}")
print(f"Promedio de ingreso por producto: ${resumen_productos['TotalIngresos'].mean():,.2f}")


--- Datos con Ingresos Calculados (primeras 5 filas) ---
        Fecha Producto  Cantidad  PrecioUnitario  IngresoTotal
0  2025-11-03   Laptop         5            1200          6000
1  2025-11-04    Mouse        10              25           250
2  2025-11-05  Teclado         8              70           560
3  2025-11-06  Monitor         3             300           900
4  2025-11-07   Webcam         6              80           480

--- Resumen por Producto (para Gerencia) ---
  Producto  TotalUnidadesVendidas  TotalIngresos
0   Laptop                     10          12000
1  Monitor                     12           3600
2    Mouse                     31            775
3  Teclado                     25           1750
4   Webcam                     15           1200

Ingreso Total General: $19,325.00

Producto con mayor ingreso: Laptop
Producto con menor ingreso: Mouse
Promedio de ingreso por producto: $3,865.00


# **Paso 4: Exportar a Excel (El Informe Final)**

Finalmente, exportamos los resultados a un archivo Excel profesional.

## ¿Por qué Excel?
- Los gerentes y clientes entienden Excel fácilmente
- Se puede editar y compartir sin necesidad de Python o Jupyter
- Permite hacer gráficos, filtros, etc. directamente en Excel

## Estructura del Excel:
- **Pestaña 1 - "Resumen_Por_Producto"**: Los números clave para la gerencia
- **Pestaña 2 - "Datos_Consolidados"**: Todos los datos crudos por si necesitan revisar

## ExcelWriter vs to_excel:
- `pd.ExcelWriter()`: Permite escribir múltiples hojas en un solo archivo
- `to_excel()`: Escribe una sola tabla en un archivo

In [8]:
# ============================================================================
# EXPORTAR RESULTADOS A EXCEL
# ============================================================================
# Definimos el nombre del archivo de salida
nombre_archivo_salida = 'Reporte_Consolidado_Ventas.xlsx'

# Usamos ExcelWriter para poder escribir en múltiples pestañas (sheets)
# 'engine='openpyxl' es la librería que usa pandas para escribir Excel modernos
with pd.ExcelWriter(nombre_archivo_salida, engine='openpyxl') as writer:
    
    # ========================================================================
    # PESTAÑA 1: RESUMEN POR PRODUCTO (Lo principal para Gerencia)
    # ========================================================================
    # to_excel() escribe el DataFrame en una pestaña específica
    # index=False: No escribe la columna de índices (0, 1, 2, ...)
    resumen_productos.to_excel(
        writer, 
        sheet_name='Resumen_Por_Producto',  # Nombre de la pestaña
        index=False                          # No escribir índices
    )
    
    # ========================================================================
    # PESTAÑA 2: DATOS CONSOLIDADOS COMPLETOS (Para auditoría y verificación)
    # ========================================================================
    # Esta pestaña tiene todos los datos crudos por si alguien quiere revisar
    # el origen de cada transacción
    df_consolidado.to_excel(
        writer,
        sheet_name='Datos_Consolidados',  # Nombre de la pestaña
        index=False                        # No escribir índices
    )

# ============================================================================
# CONFIRMACIÓN DE ÉXITO
# ============================================================================
print(f"\n{'='*60}")
print(f"✓ ¡Éxito! Reporte guardado como '{nombre_archivo_salida}'")
print(f"{'='*60}")
print(f"\nEl archivo contiene:")
print(f"  • Pestaña 1: Resumen por Producto (5 productos)")
print(f"  • Pestaña 2: Datos Consolidados (15 filas de datos crudos)")
print(f"\nPuedes abrir el archivo con:")
print(f"  • Microsoft Excel")
print(f"  • LibreOffice Calc")
print(f"  • Google Sheets")
print(f"  • Cualquier otro lector de Excel")


✓ ¡Éxito! Reporte guardado como 'Reporte_Consolidado_Ventas.xlsx'

El archivo contiene:
  • Pestaña 1: Resumen por Producto (5 productos)
  • Pestaña 2: Datos Consolidados (15 filas de datos crudos)

Puedes abrir el archivo con:
  • Microsoft Excel
  • LibreOffice Calc
  • Google Sheets
  • Cualquier otro lector de Excel
